In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd  /content/drive/'My Drive'/NYU_Spring_2021/CCM/code/eccentricity

Mounted at /content/drive
/content/drive/My Drive/NYU_Spring_2021/CCM/code/eccentricity


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import gzip
import os
import re
import sys
import tarfile

from six.moves import urllib
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

In [3]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


In [4]:
def read_tfrecord(examples, labeled=True):
    tfrecord_format = (
        {
            "image_raw": tf.io.FixedLenFeature([], tf.string),
            "label": tf.io.FixedLenFeature([], tf.int64),
        }
    )
    example= tf.io.parse_single_example(examples, tfrecord_format)
    image = tf.io.decode_raw(example["image_raw"], tf.uint8)
    image = tf.cast(image, tf.float32) * (1. / 255)
    image.set_shape(np.prod([5, 60, 60]))
    image = tf.reshape(image, [5, 60, 60, 1])
    image = image - 0.5
    label = tf.cast(example["label"], tf.int32)
    label = tf.one_hot(label, 5)
    # label = tf.keras.utils.to_categorical(y=label, num_classes=5, dtype=tf.int32)
    return image, label

In [5]:
def load_dataset(filename, labeled=True):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filename
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord, labeled=labeled), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

In [6]:
def get_dataset(filename, labeled=True):
    dataset = load_dataset(filename, labeled=labeled)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(128)
    return dataset

In [7]:
from functools import partial
AUTOTUNE = tf.data.experimental.AUTOTUNE


train_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/train_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
val_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/validation_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'

train_dataset = get_dataset(train_filename)
val_dataset = get_dataset(val_filename)
test_dataset = get_dataset(test_filename)


In [13]:
input_shape = (128,5,60,60,1)
num_classes = 5

In [14]:
model = keras.Sequential(
    [
        layers.Conv2D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape[1:]), #[128, 5, 56, 56, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 27, 27, 32]
        layers.Conv2D(filters=32, kernel_size=5, activation='relu'), # [128, 5, 23, 23, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 11, 11, 32]
        layers.Flatten(), # [128, 19360]
        layers.Dense(num_classes, activation="softmax") # [128, 5]
    ]
)

In [15]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 5, 56, 56, 32)     832       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 5, 27, 27, 32)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 23, 23, 32)     25632     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 5, 11, 11, 32)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 19360)             0         
_________________________________________________________________
dense (Dense)                (None, 5)                 96805     
Total params: 123,269
Trainable params: 123,269
Non-trainable params: 0
__________________________________________________

In [16]:
opt = keras.optimizers.Adagrad(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [17]:
epochs = 20
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
)

Epoch 1/20
212/212 [==============================] - 30s 55ms/step - loss: 0.6501 - accuracy: 0.7769 - val_loss: 0.1032 - val_accuracy: 0.9680
Epoch 2/20
212/212 [==============================] - 9s 42ms/step - loss: 0.1005 - accuracy: 0.9705 - val_loss: 0.1664 - val_accuracy: 0.9408
Epoch 3/20
212/212 [==============================] - 9s 42ms/step - loss: 0.0677 - accuracy: 0.9809 - val_loss: 0.0412 - val_accuracy: 0.9886
Epoch 4/20
212/212 [==============================] - 9s 42ms/step - loss: 0.0465 - accuracy: 0.9852 - val_loss: 0.0371 - val_accuracy: 0.9878
Epoch 5/20
212/212 [==============================] - 9s 42ms/step - loss: 0.0381 - accuracy: 0.9890 - val_loss: 0.0325 - val_accuracy: 0.9886
Epoch 6/20
212/212 [==============================] - 9s 42ms/step - loss: 0.0323 - accuracy: 0.9903 - val_loss: 0.0262 - val_accuracy: 0.9923
Epoch 7/20
212/212 [==============================] - 9s 42ms/step - loss: 0.0293 - accuracy: 0.9907 - val_loss: 0.0247 - val_accuracy: 0.990

In [18]:
score = model.evaluate(test_dataset, verbose=0)

In [19]:
score

[0.022154493257403374, 0.9926918148994446]

In [20]:
# xa, e=240px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_1_xa_240/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=240.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)



In [21]:
score = model.evaluate(test_dataset, verbose=0)
print(score)

[1.9312080144882202, 0.263905793428421]


In [22]:
# xa, e = 480px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_1_xa_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [23]:
score = model.evaluate(test_dataset, verbose=0)
print(score)

[0.02219892293214798, 0.9926918148994446]


In [24]:
# xax, e = 480px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_2_xax_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=2-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [25]:
score = model.evaluate(test_dataset, verbose=0)
print(score)

[0.022215772420167923, 0.9926918148994446]


In [26]:
# xa, e=120px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_1_120/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [27]:
score = model.evaluate(test_dataset, verbose=0)
print(score)

[1.9331629276275635, 0.24015428125858307]


## Training Dataset 2 (xax, e=360)##

In [28]:
train_filename = 'experiments/demo-experiment/datasets/gen_fl_1_120/data/train_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
val_filename = 'experiments/demo-experiment/datasets/gen_fl_1_120/data/validation_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_filename = 'experiments/demo-experiment/datasets/gen_fl_1_120/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'

train_dataset_2 = get_dataset(train_filename)
val_dataset_2 = get_dataset(val_filename)
test_dataset_2 = get_dataset(test_filename)

In [29]:
model_2 = keras.Sequential(
    [
        layers.Conv2D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape[1:]), #[128, 5, 56, 56, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 27, 27, 32]
        layers.Conv2D(filters=32, kernel_size=5, activation='relu'), # [128, 5, 23, 23, 32]
        layers.MaxPooling3D(pool_size=(1,3,3), strides=(1,2,2)), # [128, 5, 11, 11, 32]
        layers.Flatten(), # [128, 19360]
        layers.Dense(num_classes, activation="softmax") # [128, 5]
    ]
)

In [30]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 5, 56, 56, 32)     832       
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 5, 27, 27, 32)     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 5, 23, 23, 32)     25632     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 5, 11, 11, 32)     0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 19360)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 96805     
Total params: 123,269
Trainable params: 123,269
Non-trainable params: 0
________________________________________________

In [31]:
opt = keras.optimizers.Adagrad(learning_rate=0.01)
model_2.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])


In [32]:
epochs = 20
history_2 = model_2.fit(
    train_dataset_2,
    epochs=epochs,
    validation_data=val_dataset_2,
)

Epoch 1/20
212/212 [==============================] - 14s 56ms/step - loss: 1.5338 - accuracy: 0.4116 - val_loss: 0.5794 - val_accuracy: 0.7866
Epoch 2/20
212/212 [==============================] - 9s 41ms/step - loss: 0.3309 - accuracy: 0.9169 - val_loss: 0.1677 - val_accuracy: 0.9469
Epoch 3/20
212/212 [==============================] - 9s 41ms/step - loss: 0.1627 - accuracy: 0.9514 - val_loss: 0.1184 - val_accuracy: 0.9627
Epoch 4/20
212/212 [==============================] - 9s 41ms/step - loss: 0.1226 - accuracy: 0.9627 - val_loss: 0.1223 - val_accuracy: 0.9602
Epoch 5/20
212/212 [==============================] - 9s 41ms/step - loss: 0.1005 - accuracy: 0.9692 - val_loss: 0.0918 - val_accuracy: 0.9692
Epoch 6/20
212/212 [==============================] - 9s 41ms/step - loss: 0.0881 - accuracy: 0.9739 - val_loss: 0.0878 - val_accuracy: 0.9712
Epoch 7/20
212/212 [==============================] - 9s 41ms/step - loss: 0.0793 - accuracy: 0.9774 - val_loss: 0.0657 - val_accuracy: 0.978

In [36]:
score = model_2.evaluate(test_dataset_2, verbose=0)

In [37]:
print(score)

[0.034824829548597336, 0.9880227446556091]


In [38]:
# no flanker
test_filename = 'experiments/demo-experiment/datasets/gen_fl_0/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=0-te=0.0-fe=0.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)


In [39]:
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[3.105142116546631, 0.21457572281360626]


In [40]:
# xa, e = 480px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_1_xa_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [41]:
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[3.0273444652557373, 0.2204628437757492]


In [42]:
# xa, e=120px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_1_120/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=1-te=120.0-fe=360.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [43]:
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[0.034824829548597336, 0.9880227446556091]


In [44]:
# xax, e = 480px
test_filename = 'experiments/demo-experiment/datasets/gen_fl_2_xax_480/data/test_dh=120-flanker_h=120-train_ecc=0.0-ns=5-rs=False-p=even-ftype=2-te=0.0-fe=480.0-c=inf-fd=MNIST-cnorm=None-place=None.tfrecords'
test_dataset = get_dataset(test_filename)

In [46]:
score = model_2.evaluate(test_dataset, verbose=0)
print(score)

[3.097649574279785, 0.21498173475265503]
